In [59]:
import pandas as pd
import numpy as np

# Load lv1

토큰화한 텍스트를 불러오는데 2020년에 사업보고서가 나온 기업(rcp_dt = 2020..)이 존재하여 제거해주었다.

In [60]:
df = pd.read_csv("./data/preprocessing/preprocessed/df_tokenized.csv",index_col=0)
df = df.drop('str_nouns', axis=1)
df = df.drop(405,axis=0)  
df = df.drop(1779,axis=0)
df = df.drop(2037,axis=0)

In [62]:
df.head()

,연도,crp_nm,rpt_nm,rcp_no,rcp_dt,rmk,str,str_tokens
0,2019,동화약품,사업보고서 (2018.12),2.019040e+13,20190401.0,연,. 이사의 경영진단 및 분석의견num. 예측정보에 대한 주의사항당사가 본 사업보고서...,. 이사 의 경영 진단 및 분석 의견 num . 예측 정보 에 대한 주 의 사항 당...
1,2018,동화약품,[기재정정]사업보고서 (2017.12),2.018062e+13,20180618.0,연,. 이사의 경영진단 및 분석의견num. 예측정보에 대한 주의사항당사가 본 사업보고서...,. 이사 의 경영 진단 및 분석 의견 num . 예측 정보 에 대한 주 의 사항 당...
2,2017,동화약품,[기재정정]사업보고서 (2016.12),2.017041e+13,20170405.0,연,. 이사의 경영진단 및 분석의견num. 예측정보에 대한 주의사항당사가 본 사업보고서...,. 이사 의 경영 진단 및 분석 의견 num . 예측 정보 에 대한 주 의 사항 당...
3,2016,동화약품,사업보고서 (2015.12),2.016033e+13,20160330.0,연,. 이사의 경영진단 및 분석의견num. 예측정보에 대한 주의사항당사가 본 사업보고서...,. 이사 의 경영 진단 및 분석 의견 num . 예측 정보 에 대한 주 의 사항 당...
4,2015,동화약품,사업보고서 (2014.12),2.015033e+13,20150331.0,연,. 이사의 경영진단 및 분석의견num. 예측정보에 대한 주의사항당사가 본 사업보고서...,. 이사 의 경영 진단 및 분석 의견 num . 예측 정보 에 대한 주 의 사항 당...


# Filter the dataset
- There are several types of report such as restatement and first published version
- For simplifying the task, I'm going to use '사업보고서' only.  

In [72]:
naive_df = df.copy()
# naive_df = df[df.rpt_nm.map(lambda x : x.split(" ")[0]) == '사업보고서']
# naive_df.sort_values("연도",inplace=True)

# Set lv2

In [73]:
def length_adjust(str_) : 
    return (2-len(str_)) * '0' + str_

def date_convertor(date) : 
    year = length_adjust(str(date.year))
    month = length_adjust(str(date.month))
    day = length_adjust(str(date.day))
    yyyymmdd = int(year+month+day)
    return yyyymmdd

In [74]:
lv2_index = pd.date_range(start='20100101', end='20191231')
lv2_index = list(map(date_convertor,lv2_index))
lv2_columns = naive_df.crp_nm.unique()

lv2_df = \
pd.DataFrame(np.full(shape=(len(lv2_index),len(lv2_columns)),fill_value=np.nan),index=lv2_index,columns=lv2_columns)

In [75]:
lv2_df.shape
# (date_list,company_list)

(3652, 275)

# Generate the lv2

In [76]:
%%time
company_list = naive_df.crp_nm.unique()
for process,corp_name in enumerate(company_list) : 
    print("Process : {} | Total : {}".format(process+1,len(company_list)),end='\r')
    # slicing the dataset by corp_name
    tmp_df = naive_df[naive_df.crp_nm == corp_name]
    
    # set-up the time range 
    min_yr = tmp_df['연도'].min()
    max_yr = tmp_df['연도'].max()
    
    # container for saving the value 
    # those lists should be refreshed every loop.
    sim_ls = []; year_ls = []
    
    for yr in range(min_yr,max_yr) : 
        pre_df = tmp_df[tmp_df['연도'] == yr]
        next_df = tmp_df[tmp_df['연도'] == yr+1]
        publish_date = tmp_df[tmp_df['연도'] == yr+1]['rcp_dt']
        if len(pre_df) == 0 or len(next_df) == 0:
            continue
            # we don't have data
            # for example, we have 2014 but don't 2015.
            # therfore we can't measure similarity between 2014,2015.
            # so the value in lv2 at 2015 should be nan
            
        else : 
            pre_token = pre_df.str_tokens.map(lambda x : x.split(" ")).values
            next_token = next_df.str_tokens.map(lambda x : x.split(" ")).values   
            if type(pre_token[0]) == list and type(next_token[0]) == list :
                sim = len(set(next_token[0])&set(pre_token[0]))/len(set(next_token[0]))
            else : 
                 sim = len(set(next_token)&set(pre_token))/len(set(next_token))
        year_ls.append(publish_date.values[0])
        sim_ls.append(sim)
        
    assert len(year_ls) == len(sim_ls)
    lv2_df.loc[year_ls,corp_name] = sim_ls    

Wall time: 5.67 sotal : 275


# Check the lv2
- if we apply mean operation through time, we can take overall auto-regressive similarity per company.

In [77]:
lv2_df

,동화약품,우리은행,경방,삼양홀딩스,하이트진로,유한양행,CJ대한통운,하이트진로홀딩스,두산,일동홀딩스,...,한섬,일진머티리얼즈,일진디스플,코웨이,롯데쇼핑,대우조선해양,한전기술,신한지주,후성,한진중공업
20100101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20100102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20100103,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20100104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20100105,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20191227,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20191228,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20191229,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20191230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
lv2_df.mean()

동화약품      0.853813
우리은행      0.816529
경방        0.800715
삼양홀딩스     0.712607
하이트진로     0.752009
            ...   
대우조선해양    0.542492
한전기술      0.778507
신한지주      0.809471
후성        0.763435
한진중공업     0.737762
Length: 275, dtype: float64